In [1]:
from flask import Flask, jsonify, request  # Add 'request' here
from flask_cors import CORS, cross_origin
import pandas as pd    
import requests
import pickle
import shap
import io
import base64
import matplotlib.pyplot as plt
import xgboost as xgb
import logging
# import transformers
# import datasets
# import shap
# import numpy as np

final_merged_df = pd.read_csv("/home/jdineen/Documents/xaiploit/data/data.csv")
final_merged_df[final_merged_df['exploitability'] == True]

# # final_merged_df = pd.read_csv("/home/jdineen/Documents/xaiploit/data/exploit_data.csv")
# # final_merged_df[final_merged_df['cveid'] == 'cve-2008-5499']
# labels_df = pd.read_json("/home/jdineen/Documents/xaiploit/data/labels.json", lines=True)
# labels_df[labels_df['exploitability'] == True]



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


,Unnamed: 0,cveid_x,hash,crawl_date_x,publication_date_x,strings,noncomments,comments,cveid_y,crawl_date_y,publication_date_y,exploitability,exploit_code_maturity,exploit_url,exploit_evidence
250,250,cve-2018-1111,10f405927168969d1d72c7c605420b6a56b14f8ee9d457...,2018-05-18T16:47:08,2018-05-18T16:47:08,"[""#{Rex::Text.rand_text_alpha(6..12)}'&#{paylo...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2018-1111,2018-05-18T09:47:08,2018-05-18T09:47:08,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
251,251,cve-2014-9222,74cf9518c53500bd59f6c7b20d07fc9242cb75f5696cf8...,2021-02-14T02:13:39.375000,2016-11-13T09:39:26,"[""Allegro Software RomPager 'Misfortune Cookie...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2014-9222,2021-02-13T19:13:39.375000,2016-11-13T02:39:26,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
8324,8324,cve-2020-17519,0c35b476111271273f93337d873cc5ecb5ca705a5b2501...,2021-03-16T00:13:34.470000,2021-02-22T23:03:58,['Apache Flink version #{version} appears vuln...,"['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2020-17519,2021-03-15T17:13:34.470000,2021-02-22T16:03:58,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
8325,8325,cve-2013-4450,91d3ae60014af9b026ee12a324ab75b083a1a42f9c8a24...,2021-02-10T20:00:39.107000,2013-10-22T19:12:14,"['Number of requests to send', 'GEM', 'Failed ...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2013-4450,2021-02-10T13:00:39.107000,2013-10-22T12:12:14,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
8326,8326,cve-2010-4452,53899e1cabcd1afc229f65d4357a6be89f5fe14cb690cc...,2021-02-10T20:00:39.107000,2011-03-16T04:50:25,"['codebase', 'code', 'The codebase path to use...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2010-4452,2021-02-10T13:00:39.107000,2011-03-15T21:50:25,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43749,43749,cve-2021-3378,e85387e00eddd400fe5033b7a3ede53acc5f91575d3196...,2021-03-01T14:55:55,2021-03-01T14:55:55,"['Unexpected server response: #{res.code}', 'F...","['\n', '\n', '\n', '\nclass MetasploitModule <...","['##', '# This module requires Metasploit: htt...",cve-2021-3378,2021-03-01T07:55:55,2021-03-01T07:55:55,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
43750,43750,cve-2020-6207,75b7491c8733a70aa35d33182e836aaf5b154f1ce57c60...,2021-03-21T13:51:21,2021-03-21T13:51:21,"['#{@schema}#{@host}:#{@port}#{@path}', 'Start...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2020-6207,2021-03-21T06:51:21,2021-03-21T06:51:21,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
43751,43751,cve-2020-6207,959778984e1ff2e61d5a178ce9a7de9deb44e2d5f3c8a4...,2021-03-21T13:51:21,2021-03-21T13:51:21,"['#{@schema}#{@host}:#{@port}#{@path}', 'The a...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2020-6207,2021-03-21T06:51:21,2021-03-21T06:51:21,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit
43752,43752,cve-2021-22986,a08c2decb986bc570420692d25909a903c1e7a8f65f498...,2021-03-31T19:02:32,2021-03-31T19:02:32,"['Name', 'F5 iControl REST Unauthenticated SSR...","['\n', '\n', '\n', '\n\nclass MetasploitModule...","['##', '# This module requires Metasploit: htt...",cve-2021-22986,2021-03-31T12:02:32,2021-03-31T12:02:32,True,NaN,https://github.com/rapid7/metasploit-framework...,metasploit


In [30]:
final_merged_df = pd.read_csv("/home/jdineen/Documents/xaiploit/data/data.csv")
final_merged_df[final_merged_df['cveid_x'] == 'cve-2018-8540']

,Unnamed: 0,cveid_x,hash,crawl_date_x,publication_date_x,strings,noncomments,comments,cveid_y,crawl_date_y,publication_date_y,exploitability,exploit_code_maturity,exploit_url,exploit_evidence


In [8]:
import polars as pl

df = pl.read_csv("/home/jdineen/Documents/xaiploit/data/scored_data.csv")
df.filter(df["cveid"] == "cve-2008-3239")

,Unnamed: 0,cveid,hash,crawl_date,publication_date,strings,noncomments,comments,exploitability,score
i64,i64,str,str,str,str,str,str,str,str,f64
1426,1426,"""cve-2008-3239""","""4d03fb99200251…","""2021-02-10T20:…","""2009-03-23T00:…",null,"""[]""","""['============…",null,0.022452
11509,11509,"""cve-2008-3239""","""3c8d159596e02a…","""2021-02-10T19:…","""2009-03-24T00:…","""______________…","""['`', '\n<?php…","""['#!/usr/bin/p…",null,3.3957e-7
16399,16399,"""cve-2008-3239""","""8b2bbfe42b8797…","""2021-02-10T19:…","""2009-03-24T00:…",null,"""[]""","""[""\n …",null,0.022452
29523,29523,"""cve-2008-3239""","""e30661295ccf11…","""2021-02-10T19:…","""2009-03-23T00:…",null,"""[]""","""['#!/usr/bin/p…",null,0.022452
34843,34843,"""cve-2008-3239""","""74eaf19823ea34…","""2021-02-10T20:…","""2008-07-16T00:…","""http:// descri…","""['\n', '\n', '…","""['#!/usr/bin/p…",null,0.012258
36160,36160,"""cve-2008-3239""","""7b746311c07522…","""2021-02-10T20:…","""2009-03-23T00:…","""______________…","""['\n<?php\n', …","""['#!/usr/bin/p…",null,3.3957e-7
44024,44024,"""cve-2008-3239""","""aaa7752bd0997f…","""2021-02-10T19:…","""2014-07-01T00:…",null,"""[]""","""['\n …",null,0.022452


In [10]:
import torch
x = torch.rand(5, 3)
print(x)


tensor([[0.0652, 0.4840, 0.1402],
        [0.2835, 0.2525, 0.7831],
        [0.6232, 0.0233, 0.5301],
        [0.4316, 0.4958, 0.8432],
        [0.8091, 0.9510, 0.8830]])


In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
# !pip3 install torch torchvision torchaudio
# !pip3 install transformers
# !pip3 install datasets
import torch
import transformers
import datasets
import shap
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TextClassificationPipeline, pipeline, LlamaForSequenceClassification, LlamaTokenizer
import os
# token = os.environ["HF_TOKEN"]

from transformers import pipeline

pipe = pipeline("text-generation", model="TinyPixel/Llama-2-7B-bf16-sharded")

pipe("Hello, what's your name?", max_length=50, do_sample=True, temperature=0.9)
# print(list(os.environ))# 
# LlamaForSequenceClassification.from_pretrained("meta-llama/Llama-2-7b-hf")
# from transformers import AutoModelForCausalLM, AutoTokenizer
# LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# model_name = "meta-llama/Llama-2-70b-chat-hf"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# # load the emotion dataset
# dataset = pd.read_csv("data.csv")
# data = pd.DataFrame({'text':dataset['text'],'emotion':dataset['label']})

# # load the model and tokenizer
# tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)
# model = transformers.AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-emotion")#.cuda()

# # build a pipeline object to do predictions
# pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1, return_all_scores=True)
# pred

Loading checkpoint shards: 100%|██████████| 14/14 [00:05<00:00,  2.54it/s]


[{'generated_text': "Hello, what's your name?\nI'm Chandler.\nHi, Chandler.\nWelcome to our new series about personal technology.\nWe're excited to have you here.\nWe're here with"}]

In [15]:
explainer = shap.Explainer(pred)
shap_values = explainer(data['text'][:3])

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 4it [00:23, 11.86s/it]               


In [16]:
shap.plots.text(shap_values)

In [3]:
df = pd.read_csv("scored_data.csv")

# Load your DataFrame here
model = xgb.XGBClassifier()
model.load_model("model" + ".json")


with open("model_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

In [31]:
entry = df[df['cveid'] == "cve-2008-5499"]

In [73]:
sample = entry['strings'][806]

X_sample = vectorizer.transform([sample]).toarray()
explainer = shap.Explainer(model)
shap_values_sample = explainer(X_sample)
shap_values_sample
# shap.plots.text(shap_values_sample)

[17:05:24] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


.values =
array([[0.00031106, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

.base_values =
array([-3.5659475], dtype=float32)

.data =
array([[0., 0., 0., ..., 0., 0., 0.]])

In [49]:

if entry['strings'].isna().any():
    entry['strings'].fillna("", inplace=True)
print(entry['strings'][806])
# X = vectorizer.transform(entry['strings'])
# explainer = shap.Explainer(model)
# shap_values = explainer(X)
# dense_data = shap_values.data.toarray()

# # Update the data attribute of the shap_values object
# shap_values.data = dense_data

# # 2. Visualize SHAP values for the desired instance
# shap.plots.text(shap_values[1])


data exploits CVE-2008-5499.swf rb #{cli.peerhost.ljust(16)} #{self.shortname} .swf #{msg} Sending Exploit SWF clsid:D27CDB6E-AE6D-11cf-96B8-444553540000 #{obj_id} 1 1 http://download.macromedia.com/pub/shockwave/cabs/flash/swflash.cab movie #{get_resource}#{trigger_file} #{get_resource}#{trigger_file} high 1 1 #{obj_id} middle all application/x-shockwave-flash http://www.macromedia.com/go/getflashplayer #{msg} Sending HTML... msf/core Name Adobe Flash Player ActionScript Launch Command Execution Vulnerability Description License Author 0a29406d9794e4f9b30b3c5d6702c708 References CVE 2008-5499 OSVDB 50796 URL http://www.adobe.com/support/security/bulletins/apsb08-24.html URL http://www.securityfocus.com/bid/32896/exploit DefaultOptions HTTP::compression gzip HTTP::chunked Platform unix Arch Targets Automatic DisclosureDate Dec 17 2008 DefaultTarget Content-Type application/x-shockwave-flash Content-Type text/plain Content-Type text/html


In [45]:
X[0]

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [22]:
print(shap_values[0])

.values =
array([0.00092496, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ], dtype=float32)

.base_values =
-3.5659475

.data =
<11x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in Compressed Sparse Row format>


In [13]:
import numpy as np
print(np.array(entry['strings']))

[''
 'data exploits CVE-2008-5499.swf rb #{cli.peerhost.ljust(16)} #{self.shortname} .swf #{msg} Sending Exploit SWF clsid:D27CDB6E-AE6D-11cf-96B8-444553540000 #{obj_id} 1 1 http://download.macromedia.com/pub/shockwave/cabs/flash/swflash.cab movie #{get_resource}#{trigger_file} #{get_resource}#{trigger_file} high 1 1 #{obj_id} middle all application/x-shockwave-flash http://www.macromedia.com/go/getflashplayer #{msg} Sending HTML... msf/core Name Adobe Flash Player ActionScript Launch Command Execution Vulnerability Description License Author 0a29406d9794e4f9b30b3c5d6702c708 References CVE 2008-5499 OSVDB 50796 URL http://www.adobe.com/support/security/bulletins/apsb08-24.html URL http://www.securityfocus.com/bid/32896/exploit DefaultOptions HTTP::compression gzip HTTP::chunked Platform unix Arch Targets Automatic DisclosureDate Dec 17 2008 DefaultTarget Content-Type application/x-shockwave-flash Content-Type text/plain Content-Type text/html'
 'exploits CVE-2008-5499.swf rb #{cli.peer

In [32]:
# X = vectorizer.transform(entry['strings']).toarray()
# Do not extract .values here
explanation = explainer(entry['strings'])

# Use the full explanation object for plotting
shap.plots.text(explanation)

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:strings: object